<a href="https://colab.research.google.com/github/Swarn36/Finance_Related/blob/main/VaR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#VaR (Value at Risk)

How much money we can loose

In [1]:
import numpy as np
from scipy.stats import norm
import pandas as pd
import yfinance as yf
import datetime

class VaRSys:

  def __init__(self, stock, start_date, end_date):
    self.data = None
    self.stock = stock
    self.start = start_date
    self.end = end_date

  def download_data(self):
    data = {}

    ticker = yf.download(self.stock, self.start, self.end)
    data[self.stock] = ticker['Adj Close']

    return pd.DataFrame(data)

  def initialize(self):
    stock_data = self.download_data()
    stock_data['returns'] = np.log(stock_data[self.stock[0]] / stock_data[self.stock[0]].shift(1))

    self.stock_data = stock_data[1:]
    print(self.stock_data)

  def calculate_var(self, position, c): #c is the confidence level
    mu = np.mean(self.stock_data['returns'])
    sigma = np.std(self.stock_data['returns'])
    var = position * (mu - sigma*norm.ppf(1-c))
    return var

  def calculate_var_n(self, position, c, n):
    mu = np.mean(self.stock_data['returns'])
    sigma = np.std(self.stock_data['returns'])
    var = position * (mu*n - sigma*np.sqrt(n)*norm.ppf(1-c))
    return var


if __name__ == '__main__':
  start = datetime.datetime(2014, 1, 1)
  end = datetime.datetime(2018, 1, 1)
  var = VaRSys('C', start, end)
  print(var.initialize())
  print("Value at risk is: $%.2f" %var.calculate_var(1e6, 0.95))
  print("Value at risk after somedays: $%.2f" %var.calculate_var_n(1e6, 0.95, 10))

[*********************100%***********************]  1 of 1 completed
                    C   returns
Date                           
2014-01-03  42.913368  0.021388
2014-01-06  43.242855  0.007649
2014-01-07  43.540180  0.006852
2014-01-08  44.046463  0.011561
2014-01-09  44.359890  0.007091
...               ...       ...
2017-12-22  62.349060 -0.004362
2017-12-26  61.762630 -0.009450
2017-12-27  61.853497  0.001470
2017-12-28  62.010422  0.002534
2017-12-29  61.457050 -0.008964

[1006 rows x 2 columns]
None
Value at risk is: $25396.72
Value at risk after somedays: $82898.02


##Var with Monte Carlo Simulations

In [2]:
import numpy as np
from scipy.stats import norm
import pandas as pd
import yfinance as yf
import datetime

def download_data(stock, start_date, end_date):
  stock_data = {}

  ticker = yf.download(stock, start=start_date, end=end_date)
  stock_data['Adj_Close'] = ticker['Adj Close']

  return pd.DataFrame(stock_data)

class VarMonteCarlo:

  def __init__(self, S, mu, sigma, c, n, iterations):
    self.S = S
    self.mu = mu
    self.sigma = sigma
    self.c = c
    self.n = n
    self.iter = iterations

  def simulation(self):
    rand = np.random.normal(0, 1,[1, self.iter])
    stock_price = self.S * np.exp((self.mu - 0.5*self.sigma**2)*self.n + self.sigma*np.sqrt(self.n)*rand)

    stock_price = np.sort(stock_price)

    percentile = np.percentile(stock_price, (1- self.c)*100)
    print(stock_price)
    return self.S - percentile


if __name__ == '__main__':

  S = 1e6
  c = 0.95
  n = 1
  iterations = 100000

  start = datetime.datetime(2014, 1, 1)
  end = datetime.datetime(2018, 1, 1)
  citi = download_data('C', start, end)

  citi['returns'] = citi['Adj_Close'].pct_change()

  mu = np.mean(citi['returns'])
  sigma = np.std(citi['returns'])

  model = VarMonteCarlo(S, mu, sigma, c, n, iterations)
  print(f'Value at risk in monte carlo simulations: ${str(round(model.simulation(),3))}')

[*********************100%***********************]  1 of 1 completed
[[ 932898.42051741  933970.69235699  934400.85054066 ... 1061859.99361772
  1062931.18919645 1076248.16702476]]
Value at risk in monte carlo simulations: $24231.865
